# Create Heat Map for significant Cell Cycle genes

This notebook looks at the significant genes in at least one cancer in the DNA Replication pathway. Pancancer heat maps are created with circle size showing significance and color showing differences in median.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

import plot_utils as p

# Step 1: Run GSEA for significant genes in at least 1 cancer

First read in sig_pval_all_proteins.csv into a df. This csv file contains only genes with a significant p-value in at least one cancer. Then run GSEA using a list of genes from the df.

In [2]:
root = R'~\Github\WhenMutationsDontMatter\PTEN\Step_3_trans_effect\csv'
sig_df = pd.read_csv(root+R'\sig_pval_heatmap.csv')

prot_list = list(sig_df.Proteomics) # list of genes with a sig pval in >= 1 cancer
prot_enr = gp.enrichr(gene_list = prot_list, description='Tumor_partition', gene_sets='Reactome_2016', 
                       outdir='/Enrichr')

In [3]:
prot_enr.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Processing of Capped Intron-Containing Pre-mRN...,123/193,3.047452e-59,4.662602e-56,0,0,4.717289,635.599053,NUP107;NUP188;EIF4A3;HNRNPU;EFTUD2;SNRPD2;SNRP...,Reactome_2016
1,Gene Expression Homo sapiens R-HSA-74160,452/1631,6.421603e-57,4.912526e-54,0,0,2.051300,265.412919,TDRKH;RPL4;ATF2;MDC1;NUP107;HNRNPU;EHMT1;PHAX;...,Reactome_2016
2,mRNA Splicing - Major Pathway Homo sapiens R-H...,92/134,8.907630e-49,4.542892e-46,0,0,5.081918,562.262229,EIF4A3;HNRNPU;YBX1;PRPF19;USP39;ELAVL1;EFTUD2;...,Reactome_2016
3,mRNA Splicing Homo sapiens R-HSA-72172,92/144,1.076002e-44,4.115706e-42,0,0,4.729007,478.767028,EIF4A3;HNRNPU;YBX1;PRPF19;USP39;ELAVL1;EFTUD2;...,Reactome_2016
4,Infectious disease Homo sapiens R-HSA-5663205,124/348,5.502170e-26,1.683664e-23,0,0,2.637467,153.400560,RPL4;NUP107;NUP188;RPL10A;RPL9;PSMD8;RPS15;PSM...,Reactome_2016


# Step 2: Get the list of significant genes 

In [4]:
index = 20
trans = prot_enr.res2d.Genes[index]
genes = trans.split(';')
print(prot_enr.res2d.Term[index])
print('total genes:',len(genes))

DNA strand elongation Homo sapiens R-HSA-69190
total genes: 25


# Step 3: Create HeatMap

Slice out genes from the DNA Replication pathway from the df with genes sig in > 1 cancer.

In [5]:
# sig > 1 cancer
bool_df = sig_df.Proteomics.isin(genes)
plot_df = sig_df[bool_df]
len(plot_df.Proteomics.unique())

25

In [6]:
# Only include p-values a certain cutoff
a = 0.01
plot_df = plot_df.loc[plot_df['P_Value'] <= a]

In [7]:
p.plotCircleHeatMap(plot_df, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width= 800)

Loading BokehJS ...

# Step 4: Create a HeatMap with both +/- differences in median

 Read in the pos_neg_df.csv to create a df with only genes that have a pos and neg difference in median in different cancers. Slice out the genes that have a pos and neg difference in median in the pathway using the list of genes with a significant p-value in the pathway. 

In [8]:
pos_neg_df = pd.read_csv(root+R'\pos_neg_df.csv')

In [9]:
get = pos_neg_df.Proteomics.isin(genes) # bool df where True has both pos and neg
genes_pn = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn.Proteomics.unique()

array(['MCM6', 'MCM4', 'GINS3', 'RFC5', 'MCM5', 'MCM2', 'MCM7', 'MCM3',
       'RFC2', 'RFC4', 'GINS4', 'POLA2', 'GINS2'], dtype=object)

In [10]:
# Only include p-values < a certain cutoff
genes_pn = genes_pn.loc[genes_pn['P_Value'] <= a]

In [11]:
p.plotCircleHeatMap(genes_pn, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=600)

Loading BokehJS ...

# Sig in multiple cancers

In [12]:
mult_sig_df = pd.read_csv(root+R'\mult_sig_pval_heatmap.csv')

mult_sig_list = list(mult_sig_df.Proteomics.unique()) # list of genes with a sig pval in > 1 cancer
enr2 = gp.enrichr(gene_list = mult_sig_list, description='Tumor_partition', gene_sets='Reactome_2016', 
                       outdir='/Enrichr')

In [13]:
len(mult_sig_df.Proteomics.unique())

411

In [14]:
enr2.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Gene Expression Homo sapiens R-HSA-74160,101/1631,1.775164e-24,2.716001e-21,0,0,3.013392,164.796815,EHMT1;EPRS;PWP2;EFTUD2;PSMD9;SNRPD2;KHSRP;TXNL...,Reactome_2016
1,Processing of Capped Intron-Containing Pre-mRN...,36/193,3.337592e-24,2.553258e-21,0,0,9.076813,490.663374,DDX5;DHX9;DDX23;USP39;PRPF8;ELAVL1;NUP160;EFTU...,Reactome_2016
2,"Cell Cycle, Mitotic Homo sapiens R-HSA-69278",47/462,1.082883e-19,5.522702e-17,0,0,4.950443,216.183318,TOP2A;AHCTF1;PCNA;MCM7;DCTN1;NCAPG;BUB1B;SMC4;...,Reactome_2016
3,Cell Cycle Homo sapiens R-HSA-1640170,48/566,7.248901e-17,2.772705e-14,0,0,4.126796,153.364511,TOP2A;AHCTF1;PCNA;MCM7;DCTN1;NCAPG;BUB1B;SMC4;...,Reactome_2016
4,mRNA Splicing - Major Pathway Homo sapiens R-H...,24/134,4.208022e-16,1.287655e-13,0,0,8.715546,308.568418,DDX5;CPSF1;NCBP1;DHX9;NCBP2;CPSF3;CPSF2;DDX23;...,Reactome_2016


In [15]:
index2 = 5
trans2 = enr2.res2d.Genes[index2]
genes_mult = trans2.split(';')
print(enr2.res2d.Term[index2])
print('total genes:',len(genes_mult))

DNA strand elongation Homo sapiens R-HSA-69190
total genes: 14


In [16]:
bool_df = mult_sig_df.Proteomics.isin(genes_mult)
plot_df2 = mult_sig_df[bool_df]
plot_df2.Proteomics.unique()

array(['MCM6', 'MCM4', 'PCNA', 'RFC5', 'MCM5', 'MCM2', 'MCM7', 'MCM3',
       'RFC3', 'RFC2', 'RFC4', 'GINS4', 'RFC1', 'GINS2'], dtype=object)

In [17]:
# Only include p-values < a certain cutoff
plot_df2 = plot_df2.loc[plot_df2['P_Value'] <= a]

In [18]:
p.plotCircleHeatMap(plot_df2, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=700)

Loading BokehJS ...

In [19]:
get = pos_neg_df.Proteomics.isin(genes_mult) # bool df where True has both pos and neg
genes_pn_mult = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn_mult.Proteomics.unique()

array(['MCM6', 'MCM4', 'RFC5', 'MCM5', 'MCM2', 'MCM7', 'MCM3', 'RFC2',
       'RFC4', 'GINS4', 'GINS2'], dtype=object)

In [20]:
# Only include p-values < a certain cutoff
genes_pn_mult = genes_pn_mult.loc[genes_pn_mult['P_Value'] <= a]

In [21]:
p.plotCircleHeatMap(genes_pn_mult, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=600)

Loading BokehJS ...

# Check other pathways the genes (+/- and sig in mult. cancers) are part of

In [22]:
gnm = list(genes_pn_mult.Proteomics.unique())
print(gnm)
enr3 = gp.enrichr(gene_list = gnm, description='Tumor_partition', gene_sets='KEGG_2016', 
                       outdir='/Enrichr')

['MCM6', 'MCM4', 'RFC5', 'MCM5', 'MCM2', 'MCM7', 'MCM3', 'RFC2', 'RFC4', 'GINS4', 'GINS2']


In [23]:
enr3.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,DNA replication Homo sapiens hsa03030,9/36,3.662413e-24,1.073087e-21,0,0,454.545455,24529.054520,RFC5;RFC4;MCM7;RFC2;MCM3;MCM4;MCM5;MCM6;MCM2,KEGG_2016
1,Cell cycle Homo sapiens hsa04110,6/124,2.263873e-11,3.316573e-09,0,0,87.976540,2156.424556,MCM7;MCM3;MCM4;MCM5;MCM6;MCM2,KEGG_2016
2,Mismatch repair Homo sapiens hsa03430,3/23,2.178578e-07,2.127744e-05,0,0,237.154150,3637.807935,RFC5;RFC4;RFC2,KEGG_2016
3,Nucleotide excision repair Homo sapiens hsa03420,3/47,1.980407e-06,1.450648e-04,0,0,116.054159,1524.047347,RFC5;RFC4;RFC2,KEGG_2016


In [24]:
# Look at p-values 
certain_gene = 'GINS4'
all_df = pd.read_csv(root+R'\all_heatmap.csv')
gene_df = all_df.loc[all_df['Proteomics'] == certain_gene]
gene_df

,Proteomics,P_Value,Medians,Cancer
1005,GINS4,0.001694,0.742086,Gbm
11183,GINS4,0.000016,0.472815,Hnscc
25313,GINS4,0.116370,0.845900,Luad
40238,GINS4,0.645872,0.308800,Lscc
44544,GINS4,0.007736,0.679200,Brca
57652,GINS4,0.350344,-0.049301,Ov
66256,GINS4,0.051559,-0.325000,En
82045,GINS4,0.873907,0.168000,Colon
